In [1]:
# === SEKCJA 1: IMPORT I KONFIGURACJA ===
import pandas as pd
import numpy as np
import re
import warnings

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense, Dropout, BatchNormalization, Bidirectional, LeakyReLU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from IPython.display import display
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

print("Wczytywanie danych...")
# ZMIANA: Wczytujemy tylko dozwolone pliki
try:
    df_main_raw = pd.read_csv('saleflats_mazowieckie_c.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    df_slownik = pd.read_csv('slownik_finalny_zweryfikowany.csv', sep=';')
    print("Wszystkie pliki wczytane.")
except FileNotFoundError:
    print("BŁĄD: Upewnij się, że pliki 'saleflats_mazowieckie_c.csv' i 'slownik_finalny_zweryfikowany.csv' znajdują się w folderze.")
    raise

Wczytywanie danych...
Wszystkie pliki wczytane.


In [2]:
# === SEKCJA 2: PRZYGOTOWANIE DANYCH ===
print("--- Nadawanie nazw kolumnom ---")
df_main = df_main_raw.copy()
original_cols_names = [i for i in range(53)]
new_id_cols = ['WojewodztwoID', 'PowiatID', 'GminaID', 'RodzajGminyID', 'MiastoID', 'DzielnicaID', 'UlicaID']
df_main.columns = original_cols_names + new_id_cols
main_cols_map = {0: 'SaleId', 2: 'Location', 4: 'Description', 5: 'Area', 6: 'Price', 17: 'NumberOfRooms', 35: 'Floor', 36: 'Floors', 21: 'BuildingType'}
df_main.rename(columns=main_cols_map, inplace=True)

# ZMIANA: Tworzymy słownik ID -> Nazwa z naszego zweryfikowanego pliku
id_to_street_name_map = pd.Series(df_slownik.Nazwa.values, index=df_slownik.LocationID).to_dict()
print("Struktury danych przygotowane.")

print("\n--- Czyszczenie i filtrowanie ---")
numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors']
for col in numeric_features:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')

df_main['UlicaID'] = pd.to_numeric(df_main['UlicaID'], errors='coerce').fillna(0).astype(int)
df_main.dropna(subset=['Description', 'UlicaID'] + numeric_features, inplace=True)
df_main = df_main[df_main['UlicaID'] != 0].copy()

MIN_SAMPLES_PER_STREET = 10 
street_counts = df_main['UlicaID'].value_counts()
streets_to_keep = street_counts[street_counts >= MIN_SAMPLES_PER_STREET].index
df_model_ready = df_main[df_main['UlicaID'].isin(streets_to_keep)].copy()

if df_model_ready.empty:
    raise ValueError(f"Po filtracji z MIN_SAMPLES_PER_STREET = {MIN_SAMPLES_PER_STREET} nie pozostały żadne dane.")

def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())
df_model_ready['description_clean'] = df_model_ready['Description'].apply(clean_text)

print(f"Finalny zbiór danych gotowy. Wiersze: {len(df_model_ready)}, Unikalnych ulic: {df_model_ready['UlicaID'].nunique()}")

--- Nadawanie nazw kolumnom ---
Struktury danych przygotowane.

--- Czyszczenie i filtrowanie ---
Finalny zbiór danych gotowy. Wiersze: 77940, Unikalnych ulic: 1454


In [3]:
# === SEKCJA 3: PRZYGOTOWANIE DANYCH WE/WY (LABEL ENCODING DLA KATEGORII) ===
MAX_WORDS, MAX_LEN = 20000, 200
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_model_ready['description_clean'])
sequences = tokenizer.texts_to_sequences(df_model_ready['description_clean'])
X_text = pd.DataFrame(pad_sequences(sequences, maxlen=MAX_LEN), index=df_model_ready.index)

numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors']
numeric_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
X_numeric = pd.DataFrame(numeric_pipeline.fit_transform(df_model_ready[numeric_features]), index=df_model_ready.index)

# ZMIANA: Zamiast OneHotEncoder, używamy LabelEncoder dla każdej cechy kategorycznej
print("Kodowanie cech kategorycznych za pomocą Label Encoding...")
categorical_features_to_embed = ['PowiatID', 'GminaID', 'MiastoID', 'DzielnicaID', 'BuildingType']
categorical_encoders = {}
X_categorical_encoded = pd.DataFrame(index=df_model_ready.index)

for col in categorical_features_to_embed:
    df_model_ready[col] = df_model_ready[col].astype(str).fillna('missing')
    le = LabelEncoder()
    X_categorical_encoded[col] = le.fit_transform(df_model_ready[col])
    categorical_encoders[col] = le

# Zmienna celu (y) - UlicaID
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_model_ready['UlicaID'])
y_encoded_series = pd.Series(y_encoded, index=df_model_ready.index)
num_classes = len(label_encoder.classes_)
print(f"Dane przygotowane do treningu. Liczba klas (UlicaID): {num_classes}")

# Podział na zbiory
X_train_idx, X_val_idx = train_test_split(df_model_ready.index, test_size=0.2, random_state=42)
X_train_text, X_val_text = X_text.loc[X_train_idx], X_text.loc[X_val_idx]
X_train_num, X_val_num = X_numeric.loc[X_train_idx], X_numeric.loc[X_val_idx]
X_train_cat, X_val_cat = X_categorical_encoded.loc[X_train_idx], X_categorical_encoded.loc[X_val_idx]
y_train, y_val = y_encoded_series.loc[X_train_idx], y_encoded_series.loc[X_val_idx]

Kodowanie cech kategorycznych za pomocą Label Encoding...
Dane przygotowane do treningu. Liczba klas (UlicaID): 1454


In [4]:
# === SEKCJA 4: BUDOWA I TRENING MODELU Z EMBEDDINGAMI KATEGORYCZNYMI ===
from tensorflow.keras.layers import Reshape

# Wejścia modelu
input_text = Input(shape=(MAX_LEN,), name='text_input')
input_numeric = Input(shape=(X_numeric.shape[1],), name='numeric_input')
categorical_inputs = []
embedding_layers = []

# Tworzenie osobnego wejścia i warstwy Embedding dla każdej cechy kategorycznej
for col in categorical_features_to_embed:
    num_unique_values = len(categorical_encoders[col].classes_)
    embedding_dim = min(10, (num_unique_values + 1) // 2) 
    
    cat_input = Input(shape=(1,), name=f'input_{col}')
    categorical_inputs.append(cat_input)
    
    embedding = Embedding(input_dim=num_unique_values, output_dim=embedding_dim)(cat_input)
    embedding = Reshape(target_shape=(embedding_dim,))(embedding)
    embedding_layers.append(embedding)

text_embedding = Embedding(input_dim=MAX_WORDS, output_dim=128, name='text_embedding')(input_text)
lstm_layer = Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3))(text_embedding)

all_features = [lstm_layer, input_numeric] + embedding_layers
concatenated = Concatenate()(all_features)

x = Dense(512)(concatenated)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization()(x); x = Dropout(0.6)(x)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization()(x); x = Dropout(0.6)(x)
output_layer = Dense(num_classes, activation='softmax', name='output_ulica')(x)

model = Model(inputs=[input_text, input_numeric] + categorical_inputs, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Przygotowanie danych wejściowych w formie słownika
X_train_dict = {'text_input': X_train_text, 'numeric_input': X_train_num}
X_val_dict = {'text_input': X_val_text, 'numeric_input': X_val_num}
for col in categorical_features_to_embed:
    X_train_dict[f'input_{col}'] = X_train_cat[col]
    X_val_dict[f'input_{col}'] = X_val_cat[col]

callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True), ReduceLROnPlateau(monitor='val_loss', patience=3)]

print("\nRozpoczynam trening sieci neuronowej...")
history = model.fit(X_train_dict, y_train, validation_data=(X_val_dict, y_val), epochs=30, batch_size=128, callbacks=callbacks)
model.save('final_embedding_predictor.keras')

C:\Users\Ai\.conda\envs\projekt1\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 200)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_PowiatID (InputLayer)   │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_GminaID (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_MiastoID (InputLayer)   │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_DzielnicaID             │ (None, 1)                 │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_BuildingType            │ (None, 1)                 │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ text_embedding (Embedding)    │ (None, 200, 128)          │       2,560,000 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 1, 2)              │               6 │ input_PowiatID[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, 1, 1)              │               1 │ input_GminaID[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_2 (Embedding)       │ (None, 1, 10)             │             190 │ input_MiastoID[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_3 (Embedding)       │ (None, 1, 10)             │           1,060 │ input_DzielnicaID[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_4 (Embedding)       │ (None, 1, 10)             │         770,410 │ input_BuildingType[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 256)               │         263,168 │ text_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ numeric_input (InputLayer)    │ (None, 5)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape (Reshape)             │ (None, 2)                 │               0 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape_1 (Reshape)           │ (None, 1)                 │               0 │ embedding_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 4,253,953 (16.23 MB)

 Trainable params: 4,252,417 (16.22 MB)

 Non-trainable params: 1,536 (6.00 KB)


Rozpoczynam trening sieci neuronowej...
Epoch 1/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 826s 2s/step - accuracy: 0.0695 - loss: 6.3854 - val_accuracy: 0.2630 - val_loss: 4.0349 - learning_rate: 0.0010
Epoch 2/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 759s 2s/step - accuracy: 0.2479 - loss: 3.0163 - val_accuracy: 0.3174 - val_loss: 2.3839 - learning_rate: 0.0010
Epoch 3/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 836s 2s/step - accuracy: 0.2952 - loss: 2.5765 - val_accuracy: 0.3396 - val_loss: 2.2939 - learning_rate: 0.0010
Epoch 4/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 855s 2s/step - accuracy: 0.3992 - loss: 2.1458 - val_accuracy: 0.3319 - val_loss: 2.3122 - learning_rate: 0.0010
Epoch 5/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 840s 2s/step - accuracy: 0.5686 - loss: 1.5192 - val_accuracy: 0.3198 - val_loss: 2.4378 - learning_rate: 0.0010
Epoch 6/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 845s 2s/step - accuracy: 0.7252 - loss: 0.9619 - val_accuracy: 0.3038 - val_loss: 2.7423 - learning_rate: 0.0010
Epoch 7/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 8

In [5]:
# === SEKCJA 5: PREDYKCJA I ANALIZA WYNIKÓW ===
print("Przygotowywanie całego zbioru do predykcji...")
df_predict = df_model_ready.copy()

# Przygotowanie danych wejściowych
X_text_full = X_text
X_numeric_full = X_numeric
X_categorical_full_encoded = X_categorical_encoded

X_full_dict = {'text_input': X_text_full, 'numeric_input': X_numeric_full}
for col in categorical_features_to_embed:
    X_full_dict[f'input_{col}'] = X_categorical_full_encoded[col]

print("Generowanie predykcji...")
predictions_proba = model.predict(X_full_dict)

# Dekodowanie wyników
pred_encoded = np.argmax(predictions_proba, axis=1)
df_predict['Predicted_UlicaID'] = label_encoder.inverse_transform(pred_encoded)
df_predict['Predicted_Prob'] = np.max(predictions_proba, axis=1)
df_predict['Predicted_Ulica_Name'] = df_predict['Predicted_UlicaID'].apply(lambda x: id_to_street_name_map.get(x, 'Brak w słowniku'))
df_predict['Original_Ulica_Name'] = df_predict['UlicaID'].apply(lambda x: id_to_street_name_map.get(x, 'Brak w słowniku'))

print("\nPrzykładowe 25 losowych wierszy z wynikami predykcji:")
display_cols = ['SaleId', 'Location', 'Original_Ulica_Name', 'Predicted_Ulica_Name', 'Predicted_Prob']
display(df_predict[display_cols].sample(25, random_state=42))

output_filename = 'saleflats_warszawa_with_embedding_predictions.csv'
df_predict.to_csv(output_filename, index=False, sep=';')
print(f"\nFinalny DataFrame zapisano do pliku: {output_filename}")

Przygotowywanie całego zbioru do predykcji...
Generowanie predykcji...
2436/2436 ━━━━━━━━━━━━━━━━━━━━ 357s 146ms/step

Przykładowe 25 losowych wierszy z wynikami predykcji:


,SaleId,Location,Original_Ulica_Name,Predicted_Ulica_Name,Predicted_Prob
14824,312888,14,Solec,Solec,0.259970
200098,4259336,3,Obrzeżna,Obrzeżna,0.151661
140825,2881928,3,Kartograficzna,Kartograficzna,0.284749
175094,3629498,3,Siewierska,Grójecka,0.233087
100016,1691598,3,Jana iii sobieskiego,Jana iii sobieskiego,0.495564
196354,4177649,17,Lewinowska,Wyspowa,0.285392
216160,4671277,3,Płużnicka,Płużnicka,0.944367
167860,3435592,4,Marii grzegorzewskiej,Polinezyjska,0.191737
171719,3538332,17,Grójecka,Kaliska,0.264191
134738,2776737,17,Zbigniewa romaszewskiego,Zbigniewa romaszewskiego,0.359740



Finalny DataFrame zapisano do pliku: saleflats_warszawa_with_embedding_predictions.csv
